In [2]:
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [1]:
from reader import *
from string_matching_by_word import *
from string_matching_by_spaCy_NP import *
from word_embeddings_by_spaCy_NP import *
from hobbs import entry
from nltk import pos_tag

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
file_number = 'a11'

### Read Document

In [4]:
input_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/{}.input'.format(file_number)
key_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/{}.key'.format(file_number)

In [5]:
read = ReadInput(input_path)
list_of_sentences = read.getListOfSentences()
full_text = read.getText()

### Get Sentence Dict and Cluster Head Dict

In [6]:
sentence_dict = getSentenceDict(list_of_sentences)
cluster_head_dict = getClusterHeads(sentence_dict)

In [7]:
cluster_head_dict

{'X0': [2, 'F14-CRASH'],
 'X1': [4, '02-22'],
 'X2': [6, 'SERIES OF CRASHES'],
 'X3': [6, 'NAVY'],
 'X4': [6, 'ITS F-14S'],
 'X5': [13, 'one of them'],
 'X6': [13, 'Thursday'],
 'X7': [13, 'the Persian Gulf'],
 'X8': [14, 'The two-man crew'],
 'X9': [14, 'the carrier Nimitz'],
 'X10': [16, 'The three-day shutdown'],
 'X11': [16, 'the chief of naval operations'],
 'X12': [20, 'the earlier models'],
 'X13': [27, 'Navy officials'],
 'X14': [29, 'the engines'],
 'X15': [31, 'Two of the three planes that have crashed since late January'],
 'X16': [32, 'later models'],
 'X17': [37, 'an F-14D']}

### Get Coref for exact match by word

In [8]:
coref_dict_match_word = getCorefDict_match_word(sentence_dict, cluster_head_dict,90)
# coref_dict_match_word

### Get Coref for threshold match by NP

In [9]:
coref_dict_match_NP = getCorefDict_match_NP(sentence_dict, cluster_head_dict,80)
# coref_dict_match_NP

In [10]:
coref_dict_match_NP

{'X2': [[SERIES OF CRASHES, 13, ', the third crash', 83],
  [SERIES OF CRASHES, 14, 'the crash', 83],
  [SERIES OF CRASHES, 16, 'the three crashes', 100],
  [SERIES OF CRASHES, 27, 'the three recent crashes', 100],
  [SERIES OF CRASHES, 33, 'crashes', 100],
  [SERIES OF CRASHES, 34, 'the crashes', 100],
  [SERIES OF CRASHES, 40, 'the crash', 83]],
 'X4': [[ITS F-14S, 13, 'its F-14s', 100],
  [ITS F-14S, 16, 'more than 330 F-14s', 100],
  [ITS F-14S, 23, 'Ten F-14s', 100]],
 'X7': [[the Persian Gulf, 14, 'the gulf', 100],
  [the Persian Gulf, 38, 'the Persian Gulf', 100],
  [the Persian Gulf, 38, 'the gulf', 100]],
 'X8': [[The two-man crew, 15, 'The two crewmen', 100],
  [The two-man crew, 35, 'The two-man crew', 100],
  [The two-man crew, 37, 'the crew', 100]],
 'X9': [[the carrier Nimitz, 28, 'carrier-based jet fighters', 100],
  [the carrier Nimitz, 38, 'the Nimitz', 100]],
 'X10': [[The three-day shutdown, 27, 'the shutdown', 100]],
 'X11': [[the chief of naval operations,
   25,
 

### Get Coref for Word Embedding Similarity by NP

In [11]:
# coref_dict_all_sorted = getCorefDict_all_sorted(sentence_dict, cluster_head_dict,0.7)
# coref_dict_max_sentence = getCorefDict_max_of_each_sentence(sentence_dict, cluster_head_dict,0.5)

In [12]:
# coref_dict_all_sorted_top3 = get_TopN_Matches(coref_dict_all_sorted,3)

### Merging dictionaries

In [11]:
def mergeDicts(dict1, dict2):
    
    d1 = dict1.copy()
    d2 = dict2.copy()
    
    d1_keys = d1.keys()
    d2_keys = d2.keys()
    
    for key in d1_keys:
        d1[key] = list(map(lambda x: x[0:3], d1[key]))
    
    for key in d2_keys:
        d2[key] = list(map(lambda x: x[0:3], d2[key]))
    
    for key in d2_keys:
        if(key not in d1_keys):
            d1[key] = d2[key]
        else:
            for val in d2[key]:
                if(val not in d1[key]):
                    d1[key].append(val)
    
    return dict(sorted(d1.items()))

In [12]:
coref_final = mergeDicts(coref_dict_match_word, coref_dict_match_NP)

### Passing the reference dict to Hobbs

In [13]:
coref_final_with_pro = entry(list_of_sentences, cluster_head_dict, coref_final)

In [14]:
coref_final_with_pro

{'X10': [[The three-day shutdown, 27, 'the shutdown']],
 'X11': [[the chief of naval operations,
   25,
   'some more operational restrictions'],
  [the chief of naval operations, 38, 'an operation']],
 'X12': [[the earlier models, 28, 'Earlier models'],
  [the earlier models, 29, 'the first model']],
 'X13': [[Navy officials, 39, 'Navy officials']],
 'X14': [[the engines, 30, 'the engines'], [the engines, 32, 'The engines']],
 'X15': [[Two of the three planes that have crashed since late January,
   33,
   'all Navy fighter planes'],
  [Two of the three planes that have crashed since late January,
   38,
   'The F-14 that'],
  [Two of the three planes that have crashed since late January,
   39,
   'the plane'],
  [Two of the three planes that have crashed since late January,
   41,
   'The plane'],
  [Two of the three planes that have crashed since late January,
   41,
   'the plane']],
 'X2': [[SERIES OF CRASHES, 13, ', the third crash'],
  [SERIES OF CRASHES, 14, 'the crash'],
  [S

### Adding those Cluster Heads which were not predicted

In [15]:
def addNOPredsClusterHeads(cluster_head_dict, coref_final_with_pro):
    for cluster_id, cluster_val in cluster_head_dict.items():
        if(cluster_id not in coref_final_with_pro.keys()):
            coref_final_with_pro[cluster_id] = [['',int(cluster_val[0]),'', None]]
            
    return coref_final_with_pro

In [16]:
coref_final_with_pro = addNOPredsClusterHeads(cluster_head_dict, coref_final_with_pro)

In [17]:
coref_final_with_pro

{'X10': [[The three-day shutdown, 27, 'the shutdown']],
 'X11': [[the chief of naval operations,
   25,
   'some more operational restrictions'],
  [the chief of naval operations, 38, 'an operation']],
 'X12': [[the earlier models, 28, 'Earlier models'],
  [the earlier models, 29, 'the first model']],
 'X13': [[Navy officials, 39, 'Navy officials']],
 'X14': [[the engines, 30, 'the engines'], [the engines, 32, 'The engines']],
 'X15': [[Two of the three planes that have crashed since late January,
   33,
   'all Navy fighter planes'],
  [Two of the three planes that have crashed since late January,
   38,
   'The F-14 that'],
  [Two of the three planes that have crashed since late January,
   39,
   'the plane'],
  [Two of the three planes that have crashed since late January,
   41,
   'The plane'],
  [Two of the three planes that have crashed since late January,
   41,
   'the plane']],
 'X2': [[SERIES OF CRASHES, 13, ', the third crash'],
  [SERIES OF CRASHES, 14, 'the crash'],
  [S

### Taking only the head nouns

In [18]:
def takeHeadNouns(coref_final_with_pro):
    
    for cluster in coref_final_with_pro.keys():
        for i in range(0, len(coref_final_with_pro[cluster])):
            coref_final_with_pro[cluster][i][2] = coref_final_with_pro[cluster][i][2].split(' ')[-1]
                        
    return coref_final_with_pro

In [19]:
coref_final_with_pro = takeHeadNouns(coref_final_with_pro)

In [20]:
coref_final_with_pro

{'X10': [[The three-day shutdown, 27, 'shutdown']],
 'X11': [[the chief of naval operations, 25, 'restrictions'],
  [the chief of naval operations, 38, 'operation']],
 'X12': [[the earlier models, 28, 'models'],
  [the earlier models, 29, 'model']],
 'X13': [[Navy officials, 39, 'officials']],
 'X14': [[the engines, 30, 'engines'], [the engines, 32, 'engines']],
 'X15': [[Two of the three planes that have crashed since late January,
   33,
   'planes'],
  [Two of the three planes that have crashed since late January, 38, 'that'],
  [Two of the three planes that have crashed since late January, 39, 'plane'],
  [Two of the three planes that have crashed since late January, 41, 'plane'],
  [Two of the three planes that have crashed since late January, 41, 'plane']],
 'X2': [[SERIES OF CRASHES, 13, 'crash'],
  [SERIES OF CRASHES, 14, 'crash'],
  [SERIES OF CRASHES, 16, 'crashes'],
  [SERIES OF CRASHES, 27, 'crashes'],
  [SERIES OF CRASHES, 33, 'crashes'],
  [SERIES OF CRASHES, 34, 'crashes

### Print Output

In [21]:
def printOP(cluster_head_dict, coref_final_with_pro):
    
    for cluster_id, cluster_head_name in cluster_head_dict.items():

        print('<COREF ID="{}">{}</COREF>'.format(cluster_id, cluster_head_name[1]))

        coreferences = coref_final_with_pro[cluster_id]
        list_of_sent_ids = list(map(lambda x: x[1], coreferences))
        sorted_index_sent_ids = [i[0] for i in sorted(enumerate(list_of_sent_ids), key=lambda x:x[1])]
        coreferences = [coreferences[i] for i in sorted_index_sent_ids]

        for coref in coreferences:
            if(coref[0] == ''):
                continue
            print('{{{0}}}'.format(coref[1]) + ' ' + '{' + coref[2] + '}')
        print('\n', end = '')      

In [22]:
# %%capture cap --no-stderr
printOP(cluster_head_dict, coref_final_with_pro)

<COREF ID="X0">F14-CRASH</COREF>

<COREF ID="X1">02-22</COREF>

<COREF ID="X2">SERIES OF CRASHES</COREF>
{13} {crash}
{14} {crash}
{16} {crashes}
{27} {crashes}
{33} {crashes}
{34} {crashes}
{40} {crash}

<COREF ID="X3">NAVY</COREF>
{13} {Navy}
{22} {Navy}
{30} {Navy}
{33} {Navy}
{39} {Navy}
{41} {Navy}

<COREF ID="X4">ITS F-14S</COREF>
{13} {F-14s}
{16} {F-14s}
{23} {F-14s}

<COREF ID="X5">one of them</COREF>

<COREF ID="X6">Thursday</COREF>
{38} {Thursday}

<COREF ID="X7">the Persian Gulf</COREF>
{14} {gulf}
{38} {Gulf}
{38} {gulf}

<COREF ID="X8">The two-man crew</COREF>
{15} {crewmen}
{35} {crew}
{37} {crew}

<COREF ID="X9">the carrier Nimitz</COREF>
{28} {fighters}
{38} {Nimitz}

<COREF ID="X10">The three-day shutdown</COREF>
{27} {shutdown}

<COREF ID="X11">the chief of naval operations</COREF>
{25} {restrictions}
{38} {operation}

<COREF ID="X12">the earlier models</COREF>
{28} {models}
{29} {model}

<COREF ID="X13">Navy officials</COREF>
{39} {officials}

<COREF ID="X14">the en

In [387]:
with open('{}.response'.format(file_number),'w') as f:
    f.write(cap.stdout)

### Answer Key

In [103]:
# reader_o = ReadInput(key_path)
# ans = reader_o.getListOfSentences()
# ans